In [1]:
import os
from pysmiles import read_smiles
from PIL import Image
import argparse

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
from tqdm import tqdm

import torch_geometric
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv, global_max_pool, global_mean_pool, global_add_pool
from torch_geometric.data import Dataset, DataLoader
from torchvision import models

import deepchem as dc

print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Torch geometric version: {torch_geometric.__version__}")
paser = argparse.ArgumentParser()
args = paser.parse_args("")

Torch version: 1.7.1+cu110
Cuda available: True
Torch geometric version: 1.7.2


# Dataset

In [2]:
class MoleculeDataset(Dataset):
    def __init__(self, root, filename, test=False, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.filename = filename
        super(MoleculeDataset, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]
        

    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()
        #self.data = self.data.iloc[29628:]
        #print(self.data.iloc[-1])
        featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
        for index, mol in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            f = featurizer.featurize(mol["SMILES"])
            data = f[0].to_pyg_graph()
            data.smiles = mol["SMILES"]
            if not self.test:
                data.y = self._get_label(mol["target"])
            if self.test:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_test_{index}.pt'))
            else:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_{index}.pt'))
            

    def _get_label(self, label):
        if not self.test:
            label = np.asarray([label])
            return torch.tensor(label, dtype=torch.float32)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))        
        return data

In [3]:
train_dataset = MoleculeDataset(root="../data/", filename="new_train.csv")
train_dataset  = train_dataset.shuffle()

In [4]:
class Graphconv(nn.Module):
    def __init__(self, args):
        super(Graphconv, self).__init__()
        self.in_dim = args.in_dim
        self.out_dim = args.out_dim
        self.hid_dim = args.hid_dim
    
        self.num_layers = args.n_layers
        self.act = args.act
        self.use_bn = args.use_bn
        self.dropout = nn.Dropout2d(args.dropout)
        self.pool = args.pool
        
        self.layers = nn.ModuleList([])
        self.bns = nn.ModuleList([])
        for i in range(self.num_layers):
            input_dim = self.in_dim if i == 0 else self.hid_dim
            conv = GraphConv(input_dim, self.hid_dim)
            self.layers.append(conv)
            if self.use_bn:
                self.bns.append(nn.BatchNorm1d(self.hid_dim))
                
        # For graph classification
        #self.fc1 = nn.Linear(self.num_layers * self.hid_dim, self.hid_dim)
        self.fc1 = nn.Linear(self.hid_dim, self.hid_dim // 2)
        self.fc2 = nn.Linear(self.hid_dim // 2, self.out_dim)
        
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        for i, layer in enumerate(self.layers):
            x = layer(x, edge_index)
            if self.use_bn:
                x = self.bns[i](x)
            x = self.act(x)
        # Pooling layer
        x = self.pool(x, batch)
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [5]:
def train_step(batch_item, epoch, batch, training):
    label = batch_item['y']
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(batch_item)
            #print(output.shape)
            #print(label.shape)
            loss = criterion(output.view(-1), label)
        loss.backward()
        optimizer.step()
        
        return loss
    else:
        model.eval()
        with torch.no_grad():
            output = model(batch_item)
            #print(output.shape)
            #print(label.shape)            
            loss = criterion(output.view(-1), label)
            
        return loss

def experiment(train_dataloader, val_dataloader, args, model):       
    loss_plot, val_loss_plot = [], []

    for epoch in range(args.epoch):
        total_loss, total_val_loss = 0, 0

        tqdm_dataset = tqdm(enumerate(train_dataloader))
        training = True
        for batch, batch_item in tqdm_dataset:
            batch_item.to(args.device)
            batch_loss = train_step(batch_item, epoch, batch, training)
            total_loss += batch_loss

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(batch_loss.item()),
                'Total Loss' : '{:06f}'.format(total_loss/(batch+1))
            })
        loss_plot.append(total_loss/(batch+1))
        tqdm_dataset = tqdm(enumerate(val_dataloader))
        training = False
        for batch, batch_item in tqdm_dataset:
            batch_item.to(args.device)
            batch_loss = train_step(batch_item, epoch, batch, training)
            total_val_loss += batch_loss

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Val Loss': '{:06f}'.format(batch_loss.item()),
                'Total Val Loss' : '{:06f}'.format(total_val_loss/(batch+1))
            })
        val_loss_plot.append(total_val_loss/(batch+1))
        args.loss_plot = loss_plot
        args.val_loss_plot = val_loss_plot
                
        if np.min(val_loss_plot) == val_loss_plot[-1]:
            args.best_loss = np.min(val_loss_plot)
            es = 0
        else:
            es += 1
            
            if es > 15:
                print("Early stopping with best_loss: ", args.best_loss )
                break
            #args.save_path = f"./models/sage/loss{args.best_loss:.5f}-batch:{batch}-lr:{lr}-optim:{optim}-hid:{hid_dim}-n_layer:{n_layers}-act:{act}-bn:{bn}droptout:{dropout}-aggr:{aggr}-pool:{pool}.pt"
            #args.save_figure = f"./figures/sage/loss{args.best_loss:.5f}-batch:{batch}-lr:{lr}-optim:{optim}-hid:{hid_dim}-n_layer:{n_layers}-act:{act}-bn:{bn}droptout:{dropout}-aggr:{aggr}-pool:{pool}.png"
            #torch.save(model.state_dict(), args.save_path)

    return args
    
def plot_loss(args):
    plt.plot(args.loss_plot, label='train_loss')
    plt.plot(args.val_loss_plot, label='val_loss')
    plt.xlabel('epoch')
    plt.ylabel('loss(mae)')
    plt.legend()
    #plt.show()
    plt.savefig(args.save_figure)    
    plt.clf() # Clear the current figure


In [ ]:
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torch.multiprocessing as mp

args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args.shuffle = True
args.epoch = 200
args.val_size = 0.2

list_batch = [64, 128, 256, 512, 1024]
list_lr = [1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5, 5e-6, 5e-6]
list_hid_dim = [32, 64, 128, 256, 512]
list_n_layers = [1, 2, 3, 4, 5, 6]
list_act = [nn.ReLU(), nn.ELU(), nn.SELU(), nn.LeakyReLU(), nn.Tanh()]
list_bn = [True, False]
list_dropout = [0.0, 0.1, 0.2, 0.3, 0.5, 0.8]
list_aggr = ['max', 'mean']
list_pool = [global_max_pool, global_mean_pool, global_add_pool]
list_optim = ['ADAM', 'RMSProp', 'SGD']
list_l2_coef = [0.01, 0.001, 0.0001, 0]

best_model = []

for l2_coef in list_l2_coef:
    for optim in list_optim:
        for pool in list_pool:
            for aggr in list_aggr:
                for dropout in list_dropout:
                    for bn in list_bn:
                        for act in list_act:
                            for n_layers in list_n_layers:
                                for hid_dim in list_hid_dim:
                                    for batch in list_batch:
                                        for lr in list_lr:

                                            args.batch = batch
                                            args.lr = lr
                                            args.in_dim = 30
                                            args.out_dim = 1
                                            args.hid_dim = hid_dim
                                            args.n_layers = n_layers
                                            args.act = act
                                            args.use_bn = bn
                                            args.dropout = dropout
                                            args.aggr = aggr
                                            args.pool = pool
                                            args.optim = optim
                                            args.l2_coef = l2_coef
                                            
                                            train_dataloader = DataLoader(train_dataset[:24275], batch_size=args.batch, num_workers=0, shuffle=True)
                                            val_dataloader = DataLoader(train_dataset[24275:], batch_size=args.batch, num_workers=0, shuffle=True)
                                            
                                            model = Graphconv(args)
                                            model = model.to(args.device)
                                            #print("Number of parameters: ", sum(p.numel() for p in model.parameters()))
                                            #print(model)
                                            trainable_parameters = filter(lambda p: p.requires_grad, model.parameters())
                                            
                                            criterion = nn.L1Loss()
                                            if args.optim == 'ADAM':
                                                optimizer = torch.optim.Adam(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
                                            elif args.optim == 'RMSProp':
                                                optimizer = torch.optim.RMSprop(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
                                            elif args.optim == 'SGD':
                                                optimizer = torch.optim.SGD(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
                                            else:
                                                assert False, "Undefined Optimizer Type"
                                                

                                            args = experiment(train_dataloader, val_dataloader, args, model)
                                            best_model.append(args.best_loss)
                                            
                                            if np.min(best_model) == best_model[-1]:
                                                args.best_loss = np.min(best_model)
                                                args.save_path = f"./models/graphconv/loss{args.best_loss:.5f}-batch:{batch}-lr:{lr}-optim:{optim}-hid:{hid_dim}-n_layer:{n_layers}-act:{act}-bn:{bn}droptout:{dropout}-aggr:{aggr}-pool:{pool}-coef{l2_coef}.pt"
                                                args.save_figure = f"./figures/graphconv/loss{args.best_loss:.5f}-batch:{batch}-lr:{lr}-optim:{optim}-hid:{hid_dim}-n_layer:{n_layers}-act:{act}-bn:{bn}droptout:{dropout}-aggr:{aggr}-pool:{pool}-coef{l2_coef}.png"
                                                torch.save(model.state_dict(), args.save_path)
                                                plot_loss(args)

380it [00:09, 38.53it/s, Epoch=1, Loss=0.274427, Total Loss=0.271839]
95it [00:01, 49.56it/s, Epoch=1, Val Loss=0.207179, Total Val Loss=0.251918]
380it [00:10, 37.60it/s, Epoch=2, Loss=0.264835, Total Loss=0.254291]
95it [00:01, 52.75it/s, Epoch=2, Val Loss=0.260780, Total Val Loss=0.255754]
380it [00:09, 40.24it/s, Epoch=3, Loss=0.275373, Total Loss=0.250819]
95it [00:02, 45.84it/s, Epoch=3, Val Loss=0.221642, Total Val Loss=0.255613]
380it [00:09, 40.43it/s, Epoch=4, Loss=0.216683, Total Loss=0.248945]
95it [00:01, 57.11it/s, Epoch=4, Val Loss=0.229817, Total Val Loss=0.246483]
380it [00:08, 44.80it/s, Epoch=5, Loss=0.217778, Total Loss=0.246593]
95it [00:01, 60.62it/s, Epoch=5, Val Loss=0.222036, Total Val Loss=0.241520]
380it [00:09, 39.70it/s, Epoch=6, Loss=0.252392, Total Loss=0.249391]
95it [00:01, 65.59it/s, Epoch=6, Val Loss=0.243635, Total Val Loss=0.243572]
380it [00:07, 49.01it/s, Epoch=7, Loss=0.234519, Total Loss=0.247975]
95it [00:01, 52.47it/s, Epoch=7, Val Loss=0.2219

Early stopping with best_loss:  tensor(0.2372, device='cuda:0', dtype=torch.float64)


380it [00:10, 37.47it/s, Epoch=1, Loss=0.202224, Total Loss=0.269574]
95it [00:01, 51.18it/s, Epoch=1, Val Loss=0.293806, Total Val Loss=0.250409]
380it [00:10, 37.59it/s, Epoch=2, Loss=0.182258, Total Loss=0.245254]
95it [00:01, 48.73it/s, Epoch=2, Val Loss=0.247120, Total Val Loss=0.238113]
380it [00:09, 41.63it/s, Epoch=3, Loss=0.217633, Total Loss=0.239338]
95it [00:01, 54.36it/s, Epoch=3, Val Loss=0.243795, Total Val Loss=0.237630]
380it [00:08, 46.80it/s, Epoch=4, Loss=0.208049, Total Loss=0.236309]
95it [00:01, 63.33it/s, Epoch=4, Val Loss=0.301965, Total Val Loss=0.234123]
380it [00:09, 41.20it/s, Epoch=5, Loss=0.208866, Total Loss=0.233325]
95it [00:01, 60.24it/s, Epoch=5, Val Loss=0.227966, Total Val Loss=0.238819]
380it [00:07, 49.51it/s, Epoch=6, Loss=0.152240, Total Loss=0.232990]
95it [00:01, 53.99it/s, Epoch=6, Val Loss=0.236915, Total Val Loss=0.242226]
380it [00:08, 44.89it/s, Epoch=7, Loss=0.202609, Total Loss=0.232297]
95it [00:01, 52.55it/s, Epoch=7, Val Loss=0.1862

Early stopping with best_loss:  tensor(0.2214, device='cuda:0', dtype=torch.float64)


380it [00:09, 39.04it/s, Epoch=1, Loss=0.356536, Total Loss=0.299117]
95it [00:01, 54.06it/s, Epoch=1, Val Loss=0.255121, Total Val Loss=0.251693]
380it [00:09, 40.07it/s, Epoch=2, Loss=0.183911, Total Loss=0.242798]
95it [00:02, 46.40it/s, Epoch=2, Val Loss=0.207506, Total Val Loss=0.233903]
380it [00:10, 35.32it/s, Epoch=3, Loss=0.288908, Total Loss=0.226020]
95it [00:01, 52.90it/s, Epoch=3, Val Loss=0.224279, Total Val Loss=0.225841]
380it [00:09, 38.13it/s, Epoch=4, Loss=0.218899, Total Loss=0.220136]
95it [00:01, 48.89it/s, Epoch=4, Val Loss=0.260360, Total Val Loss=0.217160]
380it [00:09, 38.96it/s, Epoch=5, Loss=0.316362, Total Loss=0.215842]
95it [00:01, 56.54it/s, Epoch=5, Val Loss=0.227822, Total Val Loss=0.210544]
380it [00:09, 41.53it/s, Epoch=6, Loss=0.212754, Total Loss=0.212855]
95it [00:01, 53.65it/s, Epoch=6, Val Loss=0.227900, Total Val Loss=0.213015]
380it [00:09, 40.15it/s, Epoch=7, Loss=0.199378, Total Loss=0.210944]
95it [00:01, 50.81it/s, Epoch=7, Val Loss=0.2300

...

95it [00:06, 14.58it/s, Epoch=1, Loss=0.674736, Total Loss=0.854867]
24it [00:01, 15.27it/s, Epoch=1, Val Loss=0.596831, Total Val Loss=0.620073]
95it [00:06, 14.17it/s, Epoch=2, Loss=0.368241, Total Loss=0.430371]
24it [00:01, 15.01it/s, Epoch=2, Val Loss=0.366095, Total Val Loss=0.345423]
95it [00:06, 14.29it/s, Epoch=3, Loss=0.309013, Total Loss=0.331083]
24it [00:01, 15.26it/s, Epoch=3, Val Loss=0.289774, Total Val Loss=0.314489]
95it [00:06, 14.11it/s, Epoch=4, Loss=0.265494, Total Loss=0.308584]
24it [00:01, 15.57it/s, Epoch=4, Val Loss=0.310096, Total Val Loss=0.299314]
95it [00:06, 14.07it/s, Epoch=5, Loss=0.263425, Total Loss=0.296360]
24it [00:01, 15.61it/s, Epoch=5, Val Loss=0.273071, Total Val Loss=0.290101]
95it [00:06, 13.86it/s, Epoch=6, Loss=0.264791, Total Loss=0.288307]
24it [00:01, 16.31it/s, Epoch=6, Val Loss=0.278163, Total Val Loss=0.284020]
95it [00:06, 14.02it/s, Epoch=7, Loss=0.291590, Total Loss=0.282699]
24it [00:01, 16.64it/s, Epoch=7, Val Loss=0.296005, Tot

Early stopping with best_loss:  tensor(0.2001, device='cuda:0', dtype=torch.float64)


24it [00:05,  4.02it/s, Epoch=1, Loss=0.333251, Total Loss=0.465496]
6it [00:01,  4.59it/s, Epoch=1, Val Loss=0.374981, Total Val Loss=0.366560]
24it [00:05,  4.18it/s, Epoch=2, Loss=0.290135, Total Loss=0.302049]
6it [00:01,  4.64it/s, Epoch=2, Val Loss=0.284480, Total Val Loss=0.286025]
24it [00:05,  4.23it/s, Epoch=3, Loss=0.275321, Total Loss=0.282392]
6it [00:01,  4.31it/s, Epoch=3, Val Loss=0.279736, Total Val Loss=0.277273]
24it [00:05,  4.21it/s, Epoch=4, Loss=0.265904, Total Loss=0.274583]
6it [00:01,  4.67it/s, Epoch=4, Val Loss=0.275081, Total Val Loss=0.271181]
24it [00:05,  4.24it/s, Epoch=5, Loss=0.260547, Total Loss=0.268163]
6it [00:01,  4.13it/s, Epoch=5, Val Loss=0.267592, Total Val Loss=0.265812]
24it [00:05,  4.17it/s, Epoch=6, Loss=0.257975, Total Loss=0.262656]
6it [00:01,  4.55it/s, Epoch=6, Val Loss=0.257077, Total Val Loss=0.259732]
24it [00:05,  4.15it/s, Epoch=7, Loss=0.256767, Total Loss=0.257724]
6it [00:01,  4.22it/s, Epoch=7, Val Loss=0.258932, Total Val 

Early stopping with best_loss:  tensor(0.1935, device='cuda:0', dtype=torch.float64)


24it [00:06,  3.98it/s, Epoch=1, Loss=0.328412, Total Loss=0.654538]
6it [00:01,  3.93it/s, Epoch=1, Val Loss=0.414025, Total Val Loss=0.406559]
24it [00:05,  4.23it/s, Epoch=2, Loss=0.288738, Total Loss=0.312403]
6it [00:01,  4.09it/s, Epoch=2, Val Loss=0.280338, Total Val Loss=0.292429]
24it [00:05,  4.20it/s, Epoch=3, Loss=0.284530, Total Loss=0.289541]
6it [00:01,  4.58it/s, Epoch=3, Val Loss=0.289094, Total Val Loss=0.284568]
24it [00:05,  4.18it/s, Epoch=4, Loss=0.267943, Total Loss=0.280808]
6it [00:01,  4.29it/s, Epoch=4, Val Loss=0.268145, Total Val Loss=0.277466]
24it [00:05,  4.21it/s, Epoch=5, Loss=0.265043, Total Loss=0.275051]
6it [00:01,  4.63it/s, Epoch=5, Val Loss=0.279074, Total Val Loss=0.272087]
24it [00:05,  4.13it/s, Epoch=6, Loss=0.265812, Total Loss=0.270643]
6it [00:01,  3.99it/s, Epoch=6, Val Loss=0.276397, Total Val Loss=0.268014]
24it [00:05,  4.12it/s, Epoch=7, Loss=0.262637, Total Loss=0.266916]
6it [00:01,  4.60it/s, Epoch=7, Val Loss=0.265190, Total Val 

In [ ]:
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torch.multiprocessing as mp

args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
args.shuffle = True
args.epoch = 200
args.val_size = 0.2

args.batch = 256
args.lr = 1e-3
args.in_dim = 30
args.out_dim = 1
args.hid_dim = 128
args.n_layers = 2
args.act = nn.ELU()
args.use_bn = True
args.dropout = 0.3
args.pool = global_max_pool
args.optim = 'RMSProp'
args.l2_coef = 0.01

best_model = []

                                            
train_dataloader = DataLoader(train_dataset[:24275], batch_size=args.batch, num_workers=0, shuffle=True)
val_dataloader = DataLoader(train_dataset[24275:], batch_size=args.batch, num_workers=0, shuffle=True)

model = Graphconv(args)
model = model.to(args.device)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))
print(model)
trainable_parameters = filter(lambda p: p.requires_grad, model.parameters())

criterion = nn.L1Loss()
if args.optim == 'ADAM':
    optimizer = torch.optim.Adam(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
elif args.optim == 'RMSProp':
    optimizer = torch.optim.RMSprop(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
elif args.optim == 'SGD':
    optimizer = torch.optim.SGD(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
else:
    assert False, "Undefined Optimizer Type"


args = experiment(train_dataloader, val_dataloader, args, model)
best_model.append(args.best_loss)

if np.min(best_model) == best_model[-1]:
    args.best_loss = np.min(best_model)
    #args.save_path = f"./models/graphconv/loss{args.best_loss:.4f}-batch:{args.batch}-lr:{lr}-optim:{optim}-hid:{hid_dim}-n_layer:{n_layers}-act:{act}-bn:{bn}droptout:{dropout}-aggr:{aggr}-pool:{pool}.pt"
    #args.save_figure = f"./figures/graphconv/loss{args.best_loss:.4f}-batch:{batch}-lr:{lr}-optim:{optim}-hid:{hid_dim}-n_layer:{n_layers}-act:{act}-bn:{bn}droptout:{dropout}-aggr:{aggr}-pool:{pool}.png"
    #torch.save(model.state_dict(), args.save_path)
    #plot_loss(args)
    print(args.best_loss)

In [ ]:
test = []
a = torch.randn(3, 4)
b = torch.randn(3, 4)
bat = torch.randn(3).type(torch.LongTensor)
test.append(a)
test.append(b)
test

In [ ]:
test = torch.cat(test, dim=1)
test, test.size()

In [ ]:
test = global_max_pool(test, bat)

In [ ]:
test, test.size()

In [ ]:
## conv 거칠 때 마다의 정보를 pooling 할 건지, 최종적인 것만 pooling 할건지?

In [ ]:
import torch.nn as nn

model = nn.DataParallel(model)
model.cuda()